# Inference


## Imports


In [1]:
!ls ../input

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import os
import gc
import numpy as np
import pandas as pd
from PIL import Image

import cv2
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from tqdm import tqdm
from transformers import AutoModel, AutoConfig
from torch.optim import AdamW
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import Dinov2Model, Dinov2Config

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
print(f"PyTorch: {torch.__version__}")
print(
    f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

PyTorch: 2.9.1+cu128
Device: NVIDIA GeForce RTX 5050 Laptop GPU


In [3]:
# setting device on GPU if available, else CPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_WORKERS = 0
print('Using device:', DEVICE)
print('NUM_WORKERS:', NUM_WORKERS)
print()

# Additional Info when using cuda
if DEVICE.type == 'cuda':
    # clean GPU memory
    torch.cuda.empty_cache()
    gc.collect()

    # torch.set_float32_matmul_precision('high')

    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3, 1), 'GB')

Using device: cuda
NUM_WORKERS: 0

NVIDIA GeForce RTX 5050 Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [4]:
IS_ENSEMBLE = False

BATCH_SIZE = 16

MODELS_PATH = r'./kaggle/input/csiro-models/'
PATH_DATA = './kaggle/input/csiro-biomass'

PATH_TEST_CSV = os.path.join(PATH_DATA, 'test.csv')
PATH_TEST_IMG = os.path.join(PATH_DATA, 'test')

In [5]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if not filename.endswith('.jpg'):
            
            print(os.path.join(dirname, filename))
        # /kaggle/input/2head/pytorch/default/1/f1dinov2
        # /kaggle/input/csiro-models/blablabla.pt
        

In [6]:
model = torch.load(r'./kaggle/input/csiro-models/convnext_base.fb_in22k_ft_in1k-fold0-r2_0.7422.pt', map_location=DEVICE, weights_only=False)
model.to(DEVICE)

c:\_GitHub\CSIRO-Image2Biomass-Prediction\.venv\Lib\site-packages\torch\serialization.py:1493: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(


RecursiveScriptModule(
  original_name=InferenceWrapper
  (backbone): RecursiveScriptModule(
    original_name=ConvNeXt
    (stem): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=Conv2d)
      (1): RecursiveScriptModule(original_name=LayerNorm2d)
    )
    (stages): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=ConvNeXtStage
        (downsample): RecursiveScriptModule(original_name=Identity)
        (blocks): RecursiveScriptModule(
          original_name=Sequential
          (0): RecursiveScriptModule(
            original_name=ConvNeXtBlock
            (conv_dw): RecursiveScriptModule(original_name=Conv2d)
            (norm): RecursiveScriptModule(original_name=LayerNorm)
            (mlp): RecursiveScriptModule(
              original_name=Mlp
              (fc1): RecursiveScriptModule(original_name=Linear)
              (act): RecursiveScriptModule(original_name=GEL

In [7]:
try:
    SIZE, MEAN, STD = model.img_size, model.mean, model.std
except:
    SIZE = 224
    MEAN = [0.485, 0.456, 0.406]
    STD = [0.229, 0.224, 0.225]

### Get best model

In [20]:
# Checkpoint discovery and loading
def parse_metric_from_filename(filename: str, keyword: str) -> float:
    """Extract val_comp_metric_img from filename like ...val_comp_metric_img=0.7129.ckpt."""
    try:
        metric_part = filename.split(keyword)[-1].split('.')[:-1]
        metric_part = '.'.join(metric_part)
        return float(metric_part.replace('.ckpt', ''))
    except Exception:
        return -float('inf')

In [22]:
best_model = ''
best_score = -float('inf')

for file in os.listdir(MODELS_PATH):
    if file.endswith('.pt'):
        metric = parse_metric_from_filename(file, 'r2_')
        if metric > best_score:
            best_score = metric
            best_model = file

print(f'Best model: {best_model} with R2: {best_score}')

Best model: convnext_base.fb_in22k_ft_in1k-fold0-r2_0.7422.pt with R2: 0.7422


## Inference on Test Set

In [10]:
# Load test CSV
test_df = pd.read_csv(PATH_TEST_CSV)
test_df = test_df[~test_df['target_name'].isin(['Dry_Total_g', 'GDM_g'])]

# Pivot to one row per image
test_pivot = test_df.pivot_table(
    index='image_path',
    aggfunc='first'
).reset_index()

print(f"Test set size: {len(test_pivot)}")
print(test_pivot.head())

Test set size: 1
              image_path                   sample_id   target_name
0  test/ID1001187975.jpg  ID1001187975__Dry_Clover_g  Dry_Clover_g


In [11]:
# Create test dataset
class BiomassTestDataset(Dataset):
    """Test dataset for inference - no targets needed."""

    def __init__(self, df: pd.DataFrame, img_dir: str, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Load image
        img_path = os.path.join(
            self.img_dir, row['image_path'].replace('test/', ''))
        image = cv2.imread(img_path)

        if image is None:
            raise FileNotFoundError(f"Cannot load image: {img_path}")

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Split into left and right patches
        h, w, c = image.shape
        mid_w = w // 2

        left_patch = image[:, :mid_w, :]
        right_patch = image[:, mid_w:, :]

        # Convert to PIL
        left_pil = Image.fromarray(left_patch)
        right_pil = Image.fromarray(right_patch)

        # Apply transforms
        if self.transform:
            left_tensor = self.transform(left_pil)
            right_tensor = self.transform(right_pil)
        else:
            left_tensor = transforms.ToTensor()(left_pil)
            right_tensor = transforms.ToTensor()(right_pil)

        return {
            'left_image': left_tensor,
            'right_image': right_tensor,
            'image_id': row['image_path'].split('/')[-1].replace('.jpg', ''),
        }

In [12]:
student_val_transform = transforms.Compose([
    transforms.Resize((SIZE, SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

In [13]:
# Create test dataloader
test_dataset = BiomassTestDataset(
    df=test_pivot,
    img_dir=PATH_TEST_IMG,
    transform=student_val_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE * 2,
    shuffle=False,
    num_workers=min(NUM_WORKERS, 4),
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"Test loader created: {len(test_loader)} batches")

Test loader created: 1 batches


### TTA Inference

In [45]:
# TTA helpers
TTA_TYPES = ['id', 'hflip', 'vflip', 'hvflip']

def apply_tta(left: torch.Tensor, right: torch.Tensor, tta: str) -> tuple[torch.Tensor, torch.Tensor]:
    """Apply flip-based TTA to both patches. Input: [B, C, H, W]"""
    if tta == 'hflip':
        return torch.flip(left, dims=[3]), torch.flip(right, dims=[3])
    if tta == 'vflip':
        return torch.flip(left, dims=[2]), torch.flip(right, dims=[2])
    if tta == 'hvflip':
        return torch.flip(left, dims=[2, 3]), torch.flip(right, dims=[2, 3])
    return left, right

In [46]:
def expand_predictions(preds_3: torch.Tensor) -> torch.Tensor:
    """
    Convert [B, 3] predictions to [B, 5] submission format.
    
    Model predicts: [Dry_Green_g, Dry_Total_g, GDM_g]
    Submission needs: [Dry_Clover_g, Dry_Dead_g, Dry_Green_g, Dry_Total_g, GDM_g]
    """
    green = preds_3[:, 0]   # Dry_Green_g (predicted)
    total = preds_3[:, 1]   # Dry_Total_g (predicted)
    gdm = preds_3[:, 2]     # GDM_g (predicted)
    
    # Calculate missing targets
    dead = total - gdm      # Dry_Dead_g = Total - GDM
    clover = gdm - green    # Dry_Clover_g = GDM - Green
    
    # Return in competition order
    return torch.stack([clover, dead, green, total, gdm], dim=1)

In [47]:
def predict_model_batch(model: torch.nn.Module, batch: dict, tta_types: list[str]) -> torch.Tensor:
    """Run model over TTA variants and average. Returns [B, 5]."""
    model_preds = []
    left = batch['left_image'].to(DEVICE)
    right = batch['right_image'].to(DEVICE)
    
    for tta in tta_types:
        left_t, right_t = apply_tta(left, right, tta)
        
        # Model returns tensor [B, 3], not dict!
        preds_3 = model(left_t, right_t)  # [B, 3] = [Green, Total, GDM]
        
        # Expand to 5 targets
        preds_5 = expand_predictions(preds_3)  # [B, 5]
        model_preds.append(preds_5)
    
    # Average across TTA variants
    return torch.stack(model_preds, dim=0).mean(dim=0)  # [B, 5]

### Predict

In [48]:
all_predictions = []
all_image_ids = []

In [49]:
if IS_ENSEMBLE:
    pass
else:
    model = torch.load(
        os.path.join(MODELS_PATH, best_model),
        map_location=DEVICE,
        weights_only=False
    )
    model.to(DEVICE)
    model.eval()
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Inference"):
            # TTA predictions [B, 5]
            model_preds = predict_model_batch(model, batch, TTA_TYPES)
            
            all_predictions.append(model_preds.cpu().numpy())
            all_image_ids.extend(batch['image_id'])

    # Concatenate
    all_predictions_array = np.concatenate(all_predictions, axis=0)

print(f"Predictions shape: {all_predictions_array.shape}")
print(f"Image IDs count: {len(all_image_ids)}")

c:\_GitHub\CSIRO-Image2Biomass-Prediction\.venv\Lib\site-packages\torch\serialization.py:1493: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(
Inference: 100%|██████████| 1/1 [00:04<00:00,  4.55s/it]

Predictions shape: (1, 5)
Image IDs count: 1


In [50]:
# Format submission CSV
# Columns order: Dry_Clover_g, Dry_Dead_g, Dry_Green_g, Dry_Total_g, GDM_g
target_names = ['Dry_Clover_g', 'Dry_Dead_g',
                'Dry_Green_g', 'Dry_Total_g', 'GDM_g']

submission_rows = []

for img_idx, image_id in enumerate(all_image_ids):
    predictions = all_predictions_array[img_idx]  # [5] values for 5 targets

    for target_idx, target_name in enumerate(target_names):
        sample_id = f"{image_id}__{target_name}"
        target_value = float(predictions[target_idx])

        submission_rows.append({
            'sample_id': sample_id,
            'target': target_value
        })

# Create submission dataframe
submission_df = pd.DataFrame(submission_rows)

print(f"Submission shape: {submission_df.shape}")
print(f"Expected shape: ({len(test_pivot) * 5}, 2)")
print(submission_df.head(10))

Submission shape: (5, 2)
Expected shape: (5, 2)
                    sample_id     target
0  ID1001187975__Dry_Clover_g   4.316439
1    ID1001187975__Dry_Dead_g  23.402576
2   ID1001187975__Dry_Green_g  22.429737
3   ID1001187975__Dry_Total_g  50.148754
4         ID1001187975__GDM_g  26.746176


In [ ]:
SUBMISSION_NAME = 'submission.csv'

In [ ]:
# Save submission
submission_df.to_csv(SUBMISSION_NAME, index=False)

print(f"Submission saved to: {SUBMISSION_NAME}")

Submission saved to: submission.csv
